In [ ]:
%pip install setuptools
%pip install ydata_profiling
%pip install scikit-learn
%pip install xlrd
%pip install openpyxl
%pip install ipywidgets
%pip install statsmodels
%pip install catboost

In [ ]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from ydata_profiling import ProfileReport
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
from statsmodels.tsa.stattools import adfuller
from catboost import CatBoostRegressor
import catboost
from catboost import * 

In [ ]:
excel_data_volumes=pd.read_excel("././dataset/Объёмы перевозок.xls", skiprows=[0], header=[0,1])

In [ ]:
excel_data_volumes.sort_index(axis=1, inplace=True)
num_col=len(excel_data_volumes.columns)
left = excel_data_volumes.iloc[:, [num_col-5, num_col-4, num_col-3, num_col-2,num_col-1]]
left.columns = left.columns.get_level_values(1)
left = left.set_axis(["id", "subj_from", "subj_to", "cargo_code", "cargo_type"], axis = 1)

In [ ]:
labelencodder = LabelEncoder()
subj = pd.concat([left.loc[:, ["subj_from"]].rename(columns = {"subj_from": "subj"}), left.loc[:, ["subj_to"]].rename(columns = {"subj_to": "subj"})]).drop_duplicates(ignore_index=True)
cargo_type = left.loc[:, ["cargo_type"]].drop_duplicates(ignore_index=True)

In [ ]:
left.loc[:, ["subj_from"]] = left.loc[:, ["subj_from"]].apply(lambda x: subj[subj["subj"] == x["subj_from"]].index[0], axis=1)
left.loc[:, ["subj_to"]] = left.loc[:, ["subj_to"]].apply(lambda x: subj[subj["subj"] == x["subj_to"]].index[0], axis=1)
left.loc[:, ["cargo_type"]] = left.loc[:, ["cargo_type"]].apply(lambda x: cargo_type[cargo_type["cargo_type"] == x["cargo_type"]].index[0], axis=1)

In [ ]:
left["subj_from"] = left["subj_from"].astype("int64")
left["subj_to"] = left["subj_to"].astype("int64")
left["cargo_type"] = left["cargo_type"].astype("int64")

In [ ]:
for i in range(int((num_col-5)/2)):
    right = excel_data_volumes.iloc[:, [i*2,i*2+1]]
    date = pd.to_datetime(excel_data_volumes.iloc[:, [i*2]].columns.get_level_values(0).to_list()[0], format="%Y/%m")
    right.insert(2, ("", "date"), date)
    right.insert(3, ("", "month"), date.month)
    right.insert(4, ("", "quarter"), date.quarter)
    right.insert(5, ("", "year"), date.year)
    right.columns = right.columns.get_level_values(1)
    right = right.set_axis(["cargo_volume", "cargo_sales", "date", "month", "quarter", "year"], axis=1)
    if i == 0:
        result = pd.concat([left, right], axis = 1, sort = False)
    else:
        result = pd.concat([result, pd.concat([left, right], axis = 1, sort = False)], axis = 0, ignore_index=True, sort=False)
excel_data_volumes = result

In [ ]:
excel_data_volumes.head()

In [ ]:
excel_data_volumes.insert(11, ("month_of_period"), 0)

In [ ]:
min = excel_data_volumes["year"].min()
excel_data_volumes["month_of_period"] = excel_data_volumes["year"].apply(lambda x: x - min) + excel_data_volumes["month"] 

In [ ]:
excel_data_volumes.head(10)

In [ ]:
excel_data_volumes.insert(12, ("innactive"), 0)
excel_data_volumes.insert(13, ("active"), 0)
excel_data_volumes["innactive"] = excel_data_volumes.apply(lambda x: 1 if ((x["cargo_volume"] == 0) & (x["cargo_sales"] == 0)) else 0, axis=1)
excel_data_volumes["active"] = excel_data_volumes.apply(lambda x: 1 if ((x["cargo_volume"] > 0) & (x["cargo_sales"] > 0)) else 0, axis=1)
excel_data_volumes.drop("date", axis = 1)

In [ ]:
excel_data_volumes.to_excel("output.xlsx")

In [ ]:
excel_data_volumes.groupby(["id"]).count().iloc[:round(len(excel_data_volumes.groupby(["id"]).count())*0.2)]

In [ ]:
data_pd = excel_data_volumes

data_pd["outflow"] = 0

def calc_innactive(row):
    row_month_of_period = data_pd.iloc[row.name]["month_of_period"]
    if row_month_of_period > 1:
        subj_from = data_pd.iloc[row.name]["subj_from"]
        subj_to = data_pd.iloc[row.name]["subj_to"]
        cargo_code = data_pd.iloc[row.name]["cargo_code"]
        # cargo_type = data_pd.iloc[row.name]["cargo_type"]
        row_id = data_pd.iloc[row.name]["id"]
        row_value = data_pd.iloc[row.name]["innactive"]
        prev_row_value = data_pd.loc[(data_pd["subj_from"] == subj_from) & (data_pd["subj_to"] == subj_to) & (data_pd["cargo_code"] == cargo_code) & (data_pd["month_of_period"] == row_month_of_period -1) & (data_pd["id"] == row_id)]["innactive"].values[0]
        data_pd.iloc[row.name, data_pd.columns.get_loc("innactive")] = prev_row_value + 1
        row_value = data_pd.iloc[row.name]["innactive"]
        if row_value > 11:
            return 1
        else:
            return row_value/12
    else:
        return 0

In [ ]:
data_pd["outflow"] = data_pd.apply(lambda x: calc_innactive(x), axis=1)

In [ ]:
data_pd[data_pd["id"] == 912]

In [ ]:

data_pd = excel_data_volumes.groupby(["id", "date"], as_index=False).sum()
data_pd = data_pd.drop(data_pd.iloc[:, [2, 3, 4, 5]], axis=1)
data_pd["innactive"] = data_pd.apply(lambda x: 1 if ((x["cargo_volume"] == 0) & (x["cargo_price"] == 0)) else 0, axis=1)
data_pd["active"] = data_pd.apply(lambda x: 1 if ((x["cargo_volume"] > 0) & (x["cargo_price"] > 0)) else 0, axis=1)

data_pd["outflow"] = 0

def calc_innactive(row):
    prev_row_value = 0
    row_value = data_pd.iloc[row.name]["innactive"]
    row_id = data_pd.iloc[row.name]["id"]
    if row.name > 0:
        prev_row_id = data_pd.iloc[row.name - 1]["id"]
        if row_id == prev_row_id:
            if  row_value > 0:
                prev_row_value = data_pd.iloc[row.name - 1]["innactive"]
                data_pd.iloc[row.name, data_pd.columns.get_loc("innactive")] = prev_row_value + 1
                row_value = data_pd.iloc[row.name]["innactive"]
                if row_value > 11:
                    prev_row_value = 1
                else:
                    prev_row_value = row_value/12
            else: 
                prev_row_value = 0
    return prev_row_value

def calc_active(row):
    row_value = data_pd.iloc[row.name]["active"]
    row_id = data_pd.iloc[row.name]["id"]
    if row.name > 0:
        prev_row_id = data_pd.iloc[row.name - 1]["id"]
        if row_id == prev_row_id:
            if  row_value > 0:
                prev_row_value = data_pd.iloc[row.name - 1]["active"]
                data_pd.iloc[row.name, data_pd.columns.get_loc("active")] = prev_row_value + 1
    return 0

data_pd["outflow"] = data_pd.apply(calc_innactive, axis=1)
data_pd["innactive"] = data_pd.apply(lambda x: 1 if ((x["cargo_volume"] == 0) & (x["cargo_price"] == 0)) else 0, axis=1)
# data_pd.apply(calc_active, axis=1)

In [ ]:
dataset_fit = data_pd[data_pd["id"] == 9]
dataset_fit

In [ ]:
# Построим график продаж
plt.figure()
plt.subplot(211)
plt.plot(dataset_fit['date'], dataset_fit['cargo_volume'], color="red", marker='o', linestyle='-')
plt.title('Объемы перевозок')
plt.xlabel('Дата')
plt.ylabel('Объемы')
plt.subplot(212)
plt.plot(dataset_fit['date'], dataset_fit['cargo_price'], color="blue", marker='o', linestyle='-')
plt.title('Объемы продаж')
plt.xlabel('Дата')
plt.ylabel('Продажи')
plt.show()

In [ ]:
fit = data_pd
# fit.to_excel("output.xlsx")
l = fit.drop(labels="date", axis=1).groupby(["id"]).sum().apply(lambda x: x)
r = fit.drop(labels="date", axis=1).groupby(["id"]).outflow.last().apply(lambda x: x).rename("out")

fit_pd = pd.concat([l, r], axis=1).drop("outflow", axis=1)
fit_pd["out"] = fit_pd.apply(lambda x: 1 if (x["out"] == 1) else 0, axis=1).astype("int64")
fit_pd

In [ ]:
ProfileReport(fit_pd, minimal = True) # Формирование отчета для исследования данных

In [ ]:
fit_pd.mean() # расчет средних значений признаков для тех кто ушел и остался

In [ ]:
fit_pd.groupby(by = "out").mean() # расчет средних значений признаков для тех кто ушел и остался

In [ ]:
cor = fit_pd.corr() # матрица корреляций
fig, ax = plt.subplots(figsize=(15,5)) 
sns.heatmap(cor, annot = True, fmt = ".2f") # тепловая карта на основе матрицы корреляций

In [ ]:
scaler = StandardScaler()
data_pd[["cargo_volume", "cargo_price"]] = scaler.fit_transform(data_pd[["cargo_volume", "cargo_price"]])

cor = data_pd.corr() # матрица корреляций
fig, ax = plt.subplots(figsize=(15,5)) 
sns.heatmap(cor, annot = True, fmt = ".2f") # тепловая карта на основе матрицы корреляций

In [ ]:
scaler = StandardScaler()
fit_pd[["cargo_volume", "cargo_price"]] = scaler.fit_transform(fit_pd[["cargo_volume", "cargo_price"]])

In [ ]:
X = fit_pd.drop("out", axis = 1)
y=fit_pd["out"] # разделение данных на признаки (матрица X) и целевую переменную (y)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 0) # разделение модели на обучающую и валидационную выборку

model = LogisticRegression(max_iter = 1000)  
model.fit(X_train,y_train)
predictions = model.predict(X_test)
probabilities = model.predict_proba(X_test)[:,1]
acc = accuracy_score(y_test, predictions)
precision = precision_score (y_test, predictions)
recall = recall_score (y_test, predictions)
print(acc, precision, recall) # вывод на экран метрик, характеризующих качество модели


In [ ]:
rf_model = RandomForestClassifier(n_estimators = 100)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test) 
acc_rf = accuracy_score(y_test, y_pred)
precision_rf = precision_score (y_test, y_pred)
recall_rf = recall_score (y_test, y_pred)
print(acc_rf, precision_rf, recall_rf)# вывод на экран метрик, характеризующих качество модели

In [ ]:
sc = StandardScaler()
X_sc = sc.fit_transform(X)
linked = linkage(X_sc, method = "ward")
plt.figure(figsize=(15, 10))  
dendrogram(linked, orientation="top")
plt.show() # вывод на экран дендрограммы

In [ ]:
cl = 2 # количество кластеров выбранов соответствии с дендрограммой, а также на основе анализа нескольких вариантов
km = KMeans(n_clusters = cl)
labels = km.fit_predict(X_sc)

In [ ]:
fit_pd["claster"]= labels # добавление колонки с категориями

In [ ]:
piv = pd.pivot_table(data = fit_pd, index = "claster", columns = "out", values = "cargo_volume", aggfunc = "count" ).reset_index() # формирование сводной таблицы для анализа оттока по категориям
piv.columns = ["claster","out_0","out_1"]
piv["perc"] = piv["out_1"]/(piv["out_0"]+piv["out_1"])
piv["number"]=piv["out_1"]+piv["out_0"]
print(piv.head())

In [ ]:
cor = fit_pd.corr()
fig, ax = plt.subplots(figsize=(15,5))
sns.heatmap(cor, annot = True, fmt = ".2f")

In [ ]:
fit_pd.groupby(by = 'claster').mean()